In [145]:
from pyspark import SparkConf, SparkContext
from re import split

# Create SparkContext object
conf = SparkConf().setMaster("local").setAppName("RatingsValueCounts")
sc = SparkContext(conf=conf)

# Create lines RDD from data file
data_file_path = "../data/1800.csv"
lines = sc.textFile(data_file_path)
lines.take(2)

['ITE00100554,18000101,TMAX,-75,,,E,', 'ITE00100554,18000101,TMIN,-148,,,E,']

In [146]:
tmin_lines = lines.filter(lambda x: "TMIN" in x)
tmin_lines.take(2)

['ITE00100554,18000101,TMIN,-148,,,E,', 'EZE00100082,18000101,TMIN,-135,,,E,']

In [147]:
def parse_lines(line: str) -> tuple[str, tuple[str, int]]:
    row = split(",", line)
    station_id = row[0]
    date = row[1]
    temp = int(row[3]) * 0.1
    return (station_id, (date, temp))


daily_tmin_at_station = tmin_lines.map(parse_lines)
daily_tmin_at_station.take(3)

[('ITE00100554', ('18000101', -14.8)),
 ('EZE00100082', ('18000101', -13.5)),
 ('ITE00100554', ('18000102', -12.5))]

In [148]:
historical_tmin = daily_tmin_at_station.reduceByKey(
    lambda x, y: x if x[1] < y[1] else y
)
historical_tmin.collect()

[('ITE00100554', ('18000101', -14.8)), ('EZE00100082', ('18000101', -13.5))]

In [149]:
sc.stop()